In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
import os
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import scipy
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, f1_score
import xgboost as xgb
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
from sklearn import metrics
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
phone_gyro_df = pd.DataFrame()
filenames = os.listdir('wisdm-dataset/arff_files/phone/gyro')
filenames.pop(0)
for file in filenames:
    path = "wisdm-dataset/arff_files/phone/gyro/" + file
    data = arff.loadarff(path)
    df = pd.DataFrame(data[0])
    print("for file at " + path + "shape is: " + str(df.shape))
    frames = [df, phone_gyro_df]
    phone_gyro_df = pd.concat(frames, sort = False)
    
print(phone_gyro_df.shape)
    
phone_gyro_old_column_names = phone_gyro_df.columns

phone_gyro_new_column_names = []
for name in phone_gyro_old_column_names:
    phone_gyro_new_column_names.append(name.replace('"', ''))
    
replacement_dict = dict(zip(phone_gyro_old_column_names, phone_gyro_new_column_names))
phone_gyro_df = phone_gyro_df.rename(columns = replacement_dict)

labelencoder = LabelEncoder()
phone_gyro_df['ACTIVITY'] = labelencoder.fit_transform(phone_gyro_df['ACTIVITY'])

phone_gyro_df.pop('class')

phone_gyro_df_shuffle = shuffle(phone_gyro_df)

phone_gyro_data = phone_gyro_df_shuffle.to_numpy()

X_phone_gyro = phone_gyro_data[:,1:]
Y_phone_gyro = phone_gyro_data[:,0]
print(X_phone_gyro.shape)
print(Y_phone_gyro.shape)

x_train, x_test, y_train, y_test = train_test_split(X_phone_gyro, Y_phone_gyro, test_size = 0.2)

for file at wisdm-dataset/arff_files/phone/gyro/data_1600_gyro_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1601_gyro_phone.arffshape is: (405, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1602_gyro_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1603_gyro_phone.arffshape is: (405, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1604_gyro_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1605_gyro_phone.arffshape is: (405, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1606_gyro_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1607_gyro_phone.arffshape is: (383, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1608_gyro_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1609_gyro_phone.arffshape is: (304, 93)
for file at wisdm-dataset/arff_files/phone/gyro/data_1610_gyro_phone.a

In [3]:
space = {
    "n_estimators": hp.choice("n_estimators", [100, 150, 200, 250, 300, 350, 400, 450, 500]),
    "max_depth": hp.quniform("max_depth", 1, 15,1),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
    "min_samples_split": hp.choice("min_samples_split", [2, 3, 4, 5, 6, 7, 8, 9, 10]),
    "max_features": hp.choice("max_features", ["auto","sqrt","log2"]),
}

In [4]:
def hyperparameter_tuning(space):
    clf = RandomForestClassifier(n_estimators = int(space['n_estimators']), max_depth = space['max_depth'], 
                                 criterion = space['criterion'], min_samples_split = int(space['min_samples_split']),
                                 max_features = space['max_features'])
    acc = cross_val_score(clf, x_train, y_train,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [ ]:
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials
)

print("Best: {}".format(best))

  7%|███▏                                         | 7/100 [27:09<6:29:01, 250.98s/trial, best loss: -0.475187701342238]